# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, AtomsBase.Periodic, box=[[0.0, 2.715, 2.715], [2.715, 0.0, 2.715], [2.715, 2.715, 0.0]]u"Å")

To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.921697577774                   -0.69    6.0
  2   -7.926160566413       -2.35       -1.22    1.0
  3   -7.926838038080       -3.17       -2.37    1.8
  4   -7.926861519286       -4.63       -3.06    2.8
  5   -7.926861650755       -6.88       -3.46    1.8
  6   -7.926861673019       -7.65       -3.88    1.9
  7   -7.926861680362       -8.13       -4.29    1.2
  8   -7.926861681721       -8.87       -4.78    1.6
  9   -7.926861681862       -9.85       -5.32    1.9
 10   -7.926861681871      -11.05       -5.80    2.0
 11   -7.926861681873      -11.81       -6.65    1.6
 12   -7.926861681873      -13.14       -7.77    2.4
 13   -7.926861681873   +  -15.05       -8.47    4.0


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),

which directly yields an AbstractSystem.

In [5]:
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.921708252663                   -0.69    5.8
  2   -7.926164734677       -2.35       -1.22    1.0
  3   -7.926837542754       -3.17       -2.37    1.6
  4   -7.926861483625       -4.62       -3.03    2.8
  5   -7.926861636864       -6.81       -3.37    1.9
  6   -7.926861667757       -7.51       -3.74    1.6
  7   -7.926861680507       -7.89       -4.35    1.2
  8   -7.926861681748       -8.91       -4.80    2.0
  9   -7.926861681865       -9.93       -5.38    1.8
 10   -7.926861681872      -11.19       -6.05    2.1
 11   -7.926861681873      -12.13       -6.71    1.9
 12   -7.926861681873      -13.47       -7.31    2.1
 13   -7.926861681873      -14.35       -8.09    2.2


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [6]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.921689779249                   -0.69    5.9
  2   -7.926166842701       -2.35       -1.22    1.0
  3   -7.926840474818       -3.17       -2.37    1.5
  4   -7.926864897926       -4.61       -3.03    2.6
  5   -7.926865049911       -6.82       -3.39    2.0
  6   -7.926865079263       -7.53       -3.76    1.8
  7   -7.926865091571       -7.91       -4.34    1.2


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [7]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, AtomsBase.Periodic, box=[[0.0, 5.13, 5.13], [5.13, 0.0, 5.13], [5.13, 5.13, 0.0]]u"a₀")

Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [8]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, AtomsBase.Periodic, box=[[0.0, 5.131551291421279, 5.131551291421279], [5.131551291421279, 0.0, 5.131551291421279], [5.131551291421279, 5.131551291421279, 0.0]]u"a₀")